Step 1: Scrapping youtube video's comments using GCP's service - YouTube Data API v3.

In [1]:
import urllib.request
import urllib.parse
import json
import csv
import time
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
VIDEO_ID = "dc5S4IV_NeA"   # from the URL
MAX_RESULTS = 100          # max per page (YouTube allows up to 100)

def build_url(page_token=None):
    base = "https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        "part": "snippet",
        "videoId": VIDEO_ID,
        "key": API_KEY,
        "textFormat": "plainText",
        "maxResults": str(MAX_RESULTS),
        "order": "time"       # or "relevance"
    }
    if page_token:
        params["pageToken"] = page_token
    return base + "?" + urllib.parse.urlencode(params)

def fetch_page(page_token=None):
    url = build_url(page_token)
    with urllib.request.urlopen(url) as response:
        data = response.read().decode("utf-8")
    return json.loads(data)

def scrape_comments():
    all_comments = []
    next_page_token = None
    page = 1

    while True:
        print(f"Fetching page {page}...")
        data = fetch_page(next_page_token)

        for item in data.get("items", []):
            snippet = item["snippet"]["topLevelComment"]["snippet"]
            author = snippet.get("authorDisplayName", "")
            text = snippet.get("textDisplay", "")
            published_at = snippet.get("publishedAt", "")
            like_count = snippet.get("likeCount", 0)

            all_comments.append({
                "author": author,
                "text": text,
                "published_at": published_at,
                "likes": like_count
            })

        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break

        page += 1
        # polite pause so we don't hammer the API
        time.sleep(0.5)

    return all_comments

def save_to_csv(comments, filename="youtube_comments.csv"):
    if not comments:
        return
    fieldnames = ["author", "text", "published_at", "likes"]
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in comments:
            writer.writerow(row)


comments = scrape_comments()
print(f"Total comments fetched: {len(comments)}")
save_to_csv(comments)
print("Saved to youtube_comments.csv")

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Total comments fetched: 856
Saved to youtube_comments.csv


Step 2: Data cleaning

In [2]:
import pandas as pd
import re

# 1. Load CSV
df = pd.read_csv("youtube_comments.csv")

# ------- CLEANING FUNCTIONS -------

# Remove @username
def remove_usernames(text):
    return re.sub(r"@\w+", "", text)

# Remove URLs
def remove_urls(text):
    return re.sub(r"http\S+|www\.\S+", "", text)

# Remove emojis (optional)
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["  
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r"", text)

# Remove extra spaces, quotes, newlines
def clean_spacing(text):
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# ------- APPLY CLEANING -------

df["clean_text"] = df["text"].astype(str)

df["clean_text"] = df["clean_text"].apply(remove_usernames)
df["clean_text"] = df["clean_text"].apply(remove_urls)
df["clean_text"] = df["clean_text"].apply(remove_emojis)
df["clean_text"] = df["clean_text"].apply(clean_spacing)
df["clean_text"] = df["clean_text"].str.lower()

# 2. Drop duplicates
df = df.drop_duplicates(subset="clean_text")

# 3. Convert timestamp to datetime
df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")

# 4. Optionally remove meaningless comments (e.g., only emoji, single words)
df = df[df["clean_text"].str.len() > 3]

# Save cleaned file
df.to_csv("clean_comments.csv", index=False)

print("Cleaning complete. Saved as clean_comments.csv")
df.head(10)

Cleaning complete. Saved as clean_comments.csv


,author,text,published_at,likes,clean_text
0,@guardiannews,John Lewis ad kickstarts Christmas countdown t...,2025-11-04 09:21:26+00:00,40,john lewis ad kickstarts christmas countdown t...
1,@Jess-qy6pm,Was a black singer!!!!!,2025-11-14 22:30:54+00:00,0,was a black singer!!!!!
2,@hopeack12345,❤❤❤❤ Came here after watching the diabolical m...,2025-11-14 22:23:44+00:00,0,came here after watching the diabolical m&s ad...
3,@LessMarxMoreMises,Worst Christmas ad in history! Was it written ...,2025-11-14 21:01:06+00:00,0,worst christmas ad in history! was it written ...
4,@rosson1983,Excellent advert. As usual JL blows the other ...,2025-11-14 19:24:21+00:00,0,excellent advert. as usual jl blows the other ...
5,@jackieplatts4359,"Oh this is lovely and, yes, I’m crying 🥹",2025-11-14 17:38:26+00:00,0,"oh this is lovely and, yes, i’m crying 🥹"
6,@m535i,Well done to all those who put this amazing Jo...,2025-11-14 16:18:35+00:00,0,well done to all those who put this amazing jo...
7,@juliabroadley8411,Beautiful 😢❤,2025-11-14 14:40:39+00:00,0,beautiful
8,@666kismett,Another disappointing Christmas advert,2025-11-14 12:44:17+00:00,0,another disappointing christmas advert
9,@engerland3239,Old skoolers ❤️ ❤,2025-11-14 10:45:24+00:00,0,old skoolers


Step 3: Topic Modeling 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

N_TOPICS = 6          # tweak this after you see results
N_TOP_WORDS = 12

df = pd.read_csv("clean_comments.csv")
texts = df["clean_text"].fillna("").tolist()

# 1) TF-IDF
tfidf = TfidfVectorizer(
    max_df=0.9,
    min_df=5,           # ignore very rare words
    stop_words="english"
)
X = tfidf.fit_transform(texts)

# 2) LDA
lda = LatentDirichletAllocation(
    n_components=N_TOPICS,
    random_state=42,
    learning_method="batch"
)
topic_distributions = lda.fit_transform(X)   # shape: (n_docs, N_TOPICS)

# 3) Show top words per topic to help you interpret & label them
feature_names = tfidf.get_feature_names_out()

def print_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_indices = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_indices]
        print(f"Topic {topic_idx}: {' | '.join(top_words)}")

print_topics(lda, feature_names, N_TOP_WORDS)

# 4) Assign dominant topic per comment
df["topic_id"] = topic_distributions.argmax(axis=1)

# Optionally, after you inspect print_topics output,
# you can manually map topic_id -> human-readable label, e.g.:
topic_labels = {
    0: "nostalgia & memories",
    1: "worst / cringe / negative",
    2: "music & 90s rave",
    3: "family & fathers & sons",
    4: "representation / identity",
    5: "general praise"
}
df["topic_label"] = df["topic_id"].map(topic_labels)

df.to_csv("all_comments_with_topics.csv", index=False)
print("Saved with topics: all_comments_with_topics.csv")

Topic 0: don | nice | better | music | real | tune | just | absolutely | loved | house | did | dads
Topic 1: beautiful | brilliant | amazing | crying | advert | got | oh | wonderful | jl | club | absolutely | perfect
Topic 2: old | year | nailed | guardian | emotional | feels | right | miss | world | hits | just | boys
Topic 3: lewis | john | advert | aftersun | christmas | great | ad | time | thank | xmas | going | father
Topic 4: best | years | christmas | just | advert | seen | worst | ve | ad | people | dad | getting
Topic 5: love | family | finally | advert | white | song | bring | tears | wow | video | poor | kids
Saved with topics: all_comments_with_topics.csv


Step 4: Sentiment Analysis

In [4]:
from transformers import pipeline

df = pd.read_csv("all_comments_with_topics.csv")

sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

texts = df["clean_text"].fillna("").tolist()

labels = []
scores = []

# run in small batches so it’s not super slow
BATCH_SIZE = 32
for i in range(0, len(texts), BATCH_SIZE):
    batch = texts[i:i+BATCH_SIZE]
    results = sentiment_model(batch)
    for r in results:
        labels.append(r["label"])   # POSITIVE / NEGATIVE
        scores.append(r["score"])

df["sentiment_label"] = labels
df["sentiment_score"] = scores

df.to_csv("all_comments_with_topics_and_sentiment.csv", index=False)
print("Saved → all_comments_with_topics_and_sentiment.csv")

/Users/lasyar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/lasyar/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


Saved → all_comments_with_topics_and_sentiment.csv


In [6]:
# Load the final file with topics + sentiment
df = pd.read_csv("all_comments_with_topics_and_sentiment.csv")

# ------------------------------
# 1. Overall Sentiment Summary
# ------------------------------
print("\n==== Overall Sentiment Distribution ====\n")
print(df["sentiment_label"].value_counts(normalize=True).apply(lambda x: round(x*100, 2)))

# ------------------------------
# 2. Topic Distribution
# ------------------------------
print("\n==== Topic Distribution (Fraction of Comments) ====\n")
print(df["topic_label"].value_counts(normalize=True).apply(lambda x: round(x*100, 2)))

# ------------------------------
# 3. Sentiment Within Each Topic
# ------------------------------
print("\n==== Sentiment Within Each Topic ====\n")
sentiment_within_topics = (
    df.groupby("topic_label")["sentiment_label"]
      .value_counts(normalize=True)
      .rename("percentage")
      .mul(100)
      .round(2)
)

print(sentiment_within_topics)


==== Overall Sentiment Distribution ====

sentiment_label
POSITIVE    66.63
NEGATIVE    33.37
Name: proportion, dtype: float64

==== Topic Distribution (Fraction of Comments) ====

topic_label
family & fathers & sons      26.35
nostalgia & memories         16.75
representation / identity    15.64
general praise               15.15
worst / cringe / negative    13.67
music & 90s rave             12.44
Name: proportion, dtype: float64

==== Sentiment Within Each Topic ====

topic_label                sentiment_label
family & fathers & sons    POSITIVE           69.16
                           NEGATIVE           30.84
general praise             POSITIVE           67.48
                           NEGATIVE           32.52
music & 90s rave           POSITIVE           64.36
                           NEGATIVE           35.64
nostalgia & memories       POSITIVE           64.71
                           NEGATIVE           35.29
representation / identity  POSITIVE           58.27
            